An excellent overview of CTC can be found [here](https://distill.pub/2017/ctc/).

This is code originally from [this gist](https://gist.github.com/awni/56369a90d03953e370f3964c826ed4b0).

![ASR](https://distill.pub/2017/ctc/assets/speech_recognition.svg)

We start out by doing the imports:

In [42]:
import numpy as np
import math
import sys
import collections

Then we define a class for the CTC decoder:

In [105]:
class CTCDecoder:
    
    def __init__(self, alphabet):
        self.alphabet = alphabet
        self.NEG_INF = -float("inf")
        self.trace = False
    
    def make_new_beam(self):
        fn = lambda : (self.NEG_INF, self.NEG_INF)
        return collections.defaultdict(fn)
    
    def logsumexp(self, *args):
        """
        Stable log sum exp.
        """
        if all(a == self.NEG_INF for a in args):
                return self.NEG_INF
        a_max = max(args)
        lsp = math.log(sum(math.exp(a - a_max) for a in args))
        return a_max + lsp
    
    def decode(self, probs, beam_size=100, blank=0):
        """
        Performs inference for the given output probabilities.
    
        Arguments:
                probs: The output probabilities (e.g. post-softmax) for each
                    time step. Should be an array of shape (time x output dim).
                beam_size (int): Size of the beam to use during inference.
                blank (int): Index of the CTC blank label.
    
        Returns the output label sequence and the corresponding negative
        log-likelihood estimated by the decoder.
        """
        T, S = probs.shape
        probs = np.log(probs)
    
        # Elements in the beam are (prefix, (p_blank, p_no_blank))
        # Initialize the beam with the empty sequence, a probability of
        # 1 for ending in blank and zero for ending in non-blank
        # (in log space).
        beam = [(tuple(), (0.0, self.NEG_INF))]
    
        for t in range(T): # Loop over time
            if self.trace:
                print('t:', t, file=sys.stderr)
                
            if self.trace:
                print('BEAM:', beam, file=sys.stderr)
            # A default dictionary to store the next step candidates.
            next_beam = self.make_new_beam()
    
            for s in range(S): # Loop over vocab
                p = probs[t, s]
    
                # The variables p_b and p_nb are respectively the
                # probabilities for the prefix given that it ends in a
                # blank and does not end in a blank at this time step.
                for prefix, (p_b, p_nb) in beam: # Loop over beam
    
                    # If we propose a blank the prefix doesn't change.
                    # Only the probability of ending in blank gets updated.
                    if s == blank:
                        n_p_b, n_p_nb = next_beam[prefix]
                        n_p_b = self.logsumexp(n_p_b, p_b + p, p_nb + p)
                        next_beam[prefix] = (n_p_b, n_p_nb)
                        continue
    
                    # Extend the prefix by the new character s and add it to
                    # the beam. Only the probability of not ending in blank
                    # gets updated.
                    end_t = prefix[-1] if prefix else None
                    n_prefix = prefix + (s,)
                    n_p_b, n_p_nb = next_beam[n_prefix]
                    if s != end_t:
                        n_p_nb = self.logsumexp(n_p_nb, p_b + p, p_nb + p)
                    else:
                        # We don't include the previous probability of not ending
                        # in blank (p_nb) if s is repeated at the end. The CTC
                        # algorithm merges characters not separated by a blank.
                        n_p_nb = self.logsumexp(n_p_nb, p_b + p)
                        
                    # *NB* this would be a good place to include an LM score.
                    next_beam[n_prefix] = (n_p_b, n_p_nb)
    
                    # If s is repeated at the end we also update the unchanged
                    # prefix. This is the merging case.
                    if s == end_t:
                        if self.trace:
                            print('MERGE', s, prefix, file=sys.stderr)
                        n_p_b, n_p_nb = next_beam[prefix]
                        n_p_nb = self.logsumexp(n_p_nb, p_nb + p)
                        next_beam[prefix] = (n_p_b, n_p_nb)
    
            # Sort and trim the beam before moving on to the
            # next time-step.
            beam = sorted(next_beam.items(),
                            key=lambda x : self.logsumexp(*x[1]),
                            reverse=True)
            beam = beam[:beam_size]
            
            if self.trace:
                print('NEW BEAM:', beam, file=sys.stderr)
    
        best = beam[0]
        return best[0], -self.logsumexp(*best[1])
    
    def test(self):
        np.random.seed(3)
    
        time = 6
        output_dim = len(self.alphabet)
    
        probs = np.random.rand(time, output_dim)
        probs = probs / np.sum(probs, axis=1, keepdims=True)
    
        labels, score = self.decode(probs)
        print(labels)
        print(''.join([self.alphabet[i] for i in labels]))
        print("Score {:.3f}".format(score))
        
    def run(self, probs):
        labels, score = self.decode(probs)
        print(labels)
        print(''.join([self.alphabet[i] for i in labels]))
        print("Score {:.3f}".format(score))



Run a quick test:

In [106]:
V = [c for c in ' abcdefghijklmnopqrstuvwxyz']
dec = CTCDecoder(V)
dec.test()


(5, 14, 7)
eng
Score 13.048


Here we make an input matrix, emulating the output of the acoustic model.

In [107]:
import random, sys

C = ['c', 'c', 'a', 'a', 't', 't'] # sequence we want to output
M = [] # matrix for output 

for c in C: 
        row = []
        for v in V:
                if v == c:
                        row.append(10) # this is the best 
                else:
                        row.append(random.randint(1,5)) # a random other value
        nrow = [i/sum(row) for i in row] # normalise 
        M.append(nrow)

M = np.array(M) # numpy-ise it.

This is our TxV matrix (timesteps by vocabulary/alphabet)

In [108]:
print(M)

[[0.01298701 0.05194805 0.03896104 0.12987013 0.01298701 0.02597403
  0.03896104 0.01298701 0.06493506 0.01298701 0.01298701 0.06493506
  0.03896104 0.02597403 0.02597403 0.05194805 0.02597403 0.06493506
  0.05194805 0.02597403 0.01298701 0.03896104 0.05194805 0.03896104
  0.02597403 0.01298701 0.02597403]
 [0.06493506 0.03896104 0.05194805 0.12987013 0.01298701 0.03896104
  0.01298701 0.01298701 0.05194805 0.05194805 0.03896104 0.03896104
  0.02597403 0.01298701 0.05194805 0.01298701 0.02597403 0.03896104
  0.06493506 0.02597403 0.01298701 0.06493506 0.02597403 0.01298701
  0.03896104 0.01298701 0.02597403]
 [0.05102041 0.10204082 0.02040816 0.05102041 0.03061224 0.05102041
  0.05102041 0.01020408 0.02040816 0.02040816 0.04081633 0.03061224
  0.04081633 0.05102041 0.02040816 0.04081633 0.03061224 0.05102041
  0.03061224 0.01020408 0.03061224 0.05102041 0.02040816 0.03061224
  0.03061224 0.03061224 0.05102041]
 [0.05050505 0.1010101  0.05050505 0.03030303 0.05050505 0.04040404
  0.0303

In [109]:
dec.run(M)

(3, 1, 20)
cat
Score 10.765


In [110]:
dec.trace = True
dec.run(M)

t: 0
BEAM: [((), (0.0, -inf))]
NEW BEAM: [((3,), (-inf, -2.0412203288596382)), ((8,), (-inf, -2.7343675094195836)), ((11,), (-inf, -2.7343675094195836)), ((17,), (-inf, -2.7343675094195836)), ((1,), (-inf, -2.9575110607337933)), ((15,), (-inf, -2.9575110607337933)), ((18,), (-inf, -2.9575110607337933)), ((22,), (-inf, -2.9575110607337933)), ((2,), (-inf, -3.245193133185574)), ((6,), (-inf, -3.245193133185574)), ((12,), (-inf, -3.245193133185574)), ((21,), (-inf, -3.245193133185574)), ((23,), (-inf, -3.245193133185574)), ((5,), (-inf, -3.6506582412937383)), ((13,), (-inf, -3.6506582412937383)), ((14,), (-inf, -3.6506582412937383)), ((16,), (-inf, -3.6506582412937383)), ((19,), (-inf, -3.6506582412937383)), ((24,), (-inf, -3.6506582412937383)), ((26,), (-inf, -3.6506582412937383)), ((), (-4.343805421853684, -inf)), ((4,), (-inf, -4.343805421853684)), ((7,), (-inf, -4.343805421853684)), ((9,), (-inf, -4.343805421853684)), ((10,), (-inf, -4.343805421853684)), ((20,), (-inf, -4.343805421853

(3, 1, 20)
cat
Score 10.765


NEW BEAM: [((3, 1, 20), (-12.521903985367642, -10.954201127489839)), ((3, 21, 1, 20), (-13.49092600161623, -11.59869600202454)), ((3, 1, 7, 20), (-14.184073182176176, -11.675730575275853)), ((3, 5, 1, 20), (-13.770510863835389, -11.860570190520447)), ((3, 1, 12, 20), (-14.184073182176176, -11.826907520877993)), ((3, 18, 1, 20), (-13.720185779947876, -11.939737485301345)), ((3, 1, 16), (-13.241972080719284, -12.084510237296918)), ((3, 21, 20), (-13.266807316143813, -12.088502258566455)), ((3, 17, 1, 20), (-13.770510863835389, -11.976960883833547)), ((3, 1, 19), (-13.289878437045127, -12.105139899719479)), ((3, 10, 1, 20), (-13.901281095018586, -11.993010277107235)), ((8, 3, 1, 20), (-13.806639587923895, -12.060323048301147)), ((17, 3, 1, 20), (-13.826910552297516, -12.083216254137218)), ((11, 3, 1, 20), (-13.826910552297516, -12.085367178070983)), ((3, 26, 20), (-13.872293558145333, -12.125166242938047)), ((3, 2, 1, 20), (-14.026444237972592, -12.126204125750466)), ((3, 11, 1, 20), (-14